In [1]:
!pip install --upgrade sacrebleu sentencepiece accelerate peft bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import json
import torch
import sacrebleu
import numpy as np
from tqdm.auto import tqdm
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

2025-08-14 02:52:08.449370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755139928.615262      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755139928.669489      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


cuda


In [3]:
DATA_DIR = '/kaggle/input/mt-data' 

def create_jsonl_from_parallel_text(en_path, vi_path, output_path):
    with open(en_path, 'r', encoding='utf-8') as f_en, \
         open(vi_path, 'r', encoding='utf-8') as f_vi, \
         open(output_path, 'w', encoding='utf-8') as f_out:
        for en_line, vi_line in zip(f_en, f_vi):
            if en_line.strip() and vi_line.strip():
                record = {"translation": {"en": en_line.strip(), "vi": vi_line.strip()}}
                f_out.write(json.dumps(record, ensure_ascii=False) + '\n')
    return output_path

train_en_path = os.path.join(DATA_DIR, "train.en.txt")
train_vi_path = os.path.join(DATA_DIR, "train.vi.txt")
test_en_path = os.path.join(DATA_DIR, "test.en.txt")
test_vi_path = os.path.join(DATA_DIR, "test.vi.txt")

train_jsonl_path = "/kaggle/working/train.jsonl"
test_jsonl_path = "/kaggle/working/test.jsonl"

create_jsonl_from_parallel_text(train_en_path, train_vi_path, train_jsonl_path)
create_jsonl_from_parallel_text(test_en_path, test_vi_path, test_jsonl_path)

json_data_files = {"train": train_jsonl_path, "test": test_jsonl_path}
raw_datasets = load_dataset("json", data_files=json_data_files)

train_val_split = raw_datasets["train"].train_test_split(test_size=0.1, seed=42)
raw_datasets["train"] = train_val_split["train"]
raw_datasets["validation"] = train_val_split["test"]

print("Cấu trúc dữ liệu:")
print(raw_datasets)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Cấu trúc dữ liệu:
DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 450000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 50000
    })
})


In [4]:
model_checkpoint = "Helsinki-NLP/opus-mt-vi-en"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_checkpoint,
    quantization_config=bnb_config,
    device_map={"":0} 
)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

In [5]:
# --- Đánh giá Baseline ---
def evaluate_model(model_to_eval, tokenizer, dataset_to_eval, batch_size=32):
    input_texts = [item['translation']['vi'] for item in dataset_to_eval]
    reference_texts = [[item['translation']['en']] for item in dataset_to_eval]
    predictions = []
    
    for i in tqdm(range(0, len(input_texts), batch_size), desc="Đang dịch tập test..."):
        batch = input_texts[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        
        generated_ids = model.generate(
            **inputs, 
            max_new_tokens=128,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
        
        batch_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        if i == 0:
            print(batch)
            print(batch_preds)
            print(reference_texts[i:i + batch_size])
        predictions.extend(batch_preds)
    
    bleu_score = sacrebleu.corpus_bleu(predictions, reference_texts)
    return bleu_score.score

print("Bắt đầu đánh giá BASELINE...")
baseline_bleu = evaluate_model(model, tokenizer, raw_datasets['test'])
print(f"\nInitial BLEU: {baseline_bleu:.2f}")

Bắt đầu đánh giá BASELINE...


Đang dịch tập test...:   0%|          | 0/94 [00:00<?, ?it/s]

['Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017', 'Mô tả thực trạng kiến thức, thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố liên quan tại tỉnh Viêng Chăn, Cộng hoà Dân chủ Nhân dân Lào năm 2017.', 'Phương pháp: Thiết kế nghiên mô tả cắt ngang được thực hiện trên 928 người trưởng thành có thẻ bảo hiểm y tế tại 2 huyện Phone Hong và Keo Oudom, tỉnh Viêng Chăn.', 'Kết quả: Tỷ lệ người biết được khám chữa bệnh (KCB) miễn phí tại nơi đăng ký ban đầu chiếm 44,5%, được cung cấp thông tin về bảo hiểm y tế (BHYT) chiếm 34,8%.', 'Tỷ lệ người có thẻ BHYT thực hành khám chữa bệnh đúng nơi đăng ký KCB ban đầu chiếm 61,8%.', 'Tỷ lệ người có thẻ BHYT sử dụng thẻ để lấy thuốc cho người khác khá cao (20,1%).', 'Các yếu tố khoảng cách từ nhà đến cơ sở y tế, thời gian tham gia BHYT và được

In [6]:
max_length = 128

def preprocess_function(examples):
    inputs = [ex['vi'] for ex in examples["translation"]]
    targets = [ex['en'] for ex in examples["translation"]]
    
    model_inputs = tokenizer(
        inputs, 
        text_target=targets, 
        max_length=max_length, 
        truncation=True
    )
    return model_inputs

tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=["translation"] 
)

Map:   0%|          | 0/450000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 589,824 || all params: 72,766,976 || trainable%: 0.8106


In [8]:
# Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Hàm tính metrics
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = sacrebleu.corpus_bleu(decoded_preds, decoded_labels)
    return {"bleu": result.score}

# Tham số huấn luyện
training_args = Seq2SeqTrainingArguments(
    output_dir="opus-mt-vi-en-medical-finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    predict_with_generate=True,
    fp16=False,
    bf16=True,
    learning_rate=3e-4,
    num_train_epochs=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none",
)

# Khởi tạo Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/tmp/ipykernel_19/4164038772.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Bleu
1,3.026500,2.720382,51.924475


TrainOutput(global_step=28125, training_loss=3.101011286892361, metrics={'train_runtime': 21376.6133, 'train_samples_per_second': 21.051, 'train_steps_per_second': 1.316, 'total_flos': 9029409012449280.0, 'train_loss': 3.101011286892361, 'epoch': 1.0})

In [9]:
finetuned_bleu = evaluate_model(trainer.model, tokenizer, raw_datasets['test'])

print(f"BLEU before finetuning: {baseline_bleu:.2f}")
print(f"BLEU after finetuning : {finetuned_bleu:.2f}")
improvement = finetuned_bleu - baseline_bleu
print(f"Improvement: +{improvement:.2f} BLEU")

# Lưu adapter LoRA
output_dir = "adapter"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Đang dịch tập test...:   0%|          | 0/94 [00:00<?, ?it/s]

['Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017', 'Mô tả thực trạng kiến thức, thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố liên quan tại tỉnh Viêng Chăn, Cộng hoà Dân chủ Nhân dân Lào năm 2017.', 'Phương pháp: Thiết kế nghiên mô tả cắt ngang được thực hiện trên 928 người trưởng thành có thẻ bảo hiểm y tế tại 2 huyện Phone Hong và Keo Oudom, tỉnh Viêng Chăn.', 'Kết quả: Tỷ lệ người biết được khám chữa bệnh (KCB) miễn phí tại nơi đăng ký ban đầu chiếm 44,5%, được cung cấp thông tin về bảo hiểm y tế (BHYT) chiếm 34,8%.', 'Tỷ lệ người có thẻ BHYT thực hành khám chữa bệnh đúng nơi đăng ký KCB ban đầu chiếm 61,8%.', 'Tỷ lệ người có thẻ BHYT sử dụng thẻ để lấy thuốc cho người khác khá cao (20,1%).', 'Các yếu tố khoảng cách từ nhà đến cơ sở y tế, thời gian tham gia BHYT và được

('adapter/tokenizer_config.json',
 'adapter/special_tokens_map.json',
 'adapter/vocab.json',
 'adapter/source.spm',
 'adapter/target.spm',
 'adapter/added_tokens.json')